In [1]:
row_sample_rate = 1
country_min_sample_cutoff = 1200
DUMMY_NA = True
FILL="Mean"
cutoffs = [5000, 2500, 1000, 500, 250, 100]


In [2]:
import numpy as np
import pandas as pd
import time
import pickle

from helpers import clone_drop_and_convert, clone_and_convert_multiple_choice_columns, clone_impute_data, perf_measures, round_decision, reduce_dataframe_using_min_non_null, get_coefficent_weights
from models import apply_linear_regression_model
from predict_country import predict_country

# Developer note: If you modify the modules above, the changes to those modules will not be reflected until you 
# re-execute this cell, specifically the reload commands
# TODO this doesn't work yet
# import reload
# reload(clone_drop_and_convert)
# reload(models)

pd.options.display.max_rows = 2000
pd.options.display.max_columns = 1000
pd.options.display.max_colwidth = 255

script_start = time.time()

raw_df = pd.read_csv('./assets/survey_results_public.csv')
schema = pd.read_csv('./assets/survey_results_schema.csv')

In [3]:
start = time.time()

df = clone_and_convert_multiple_choice_columns( \
         clone_drop_and_convert( \
             raw_df.sample(frac=row_sample_rate, axis=0) \
         ) \
     )
end = time.time()
print('convert multi choice time %ss' % round(end - start, 3))
print("convert multi choice dataset has %s rows and %s columns" % (df.shape[0], df.shape[1]))

full_country = df

start = time.time()
full_country_imputed = clone_impute_data(full_country, dummy_na=True, fill_technique=FILL)
end = time.time()
print('full_country impute time %ss' % round(end - start, 3))
print("full country imputed dataset has %s rows and %s columns" % (full_country_imputed.shape[0], full_country_imputed.shape[1]))

convert multi choice time 39.08s
convert multi choice dataset has 64461 rows and 369 columns
full_country impute time 0.737s
full country imputed dataset has 64461 rows and 454 columns


In [4]:
countries = full_country['Country'].value_counts()
countries_with_enough_samples = countries[countries > (country_min_sample_cutoff * row_sample_rate)].index.values
countries_with_enough_samples

array(['United States', 'India', 'United Kingdom', 'Germany', 'Canada',
       'France', 'Brazil', 'Netherlands', 'Poland', 'Australia'],
      dtype=object)

In [5]:
results = []
for country in countries_with_enough_samples:
    country_results = predict_country( \
        rowset_label='full_country_%s' % country, \
        columnset_label='all_numeric_categerical', \
        imputed_df=full_country_imputed, \
        cutoffs=cutoffs, \
        country_to_classify=country, \
        row_sample_rate=row_sample_rate \
    )
    results.extend(country_results)

model_type=linear_regression rowset_label=full_country_United States, columnset_label=all_numeric_categerical, cutoff=5000 yield columns=293, ACC=0.8302911215678164, in duration=0.6762s
model_type=random_forest rowset_label=full_country_United States, columnset_label=all_numeric_categerical, cutoff=5000 yield columns=293, ACC=0.8334970784425255, in duration=9.412s


/opt/conda/lib/python3.9/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


model_type=linear_svm rowset_label=full_country_United States, columnset_label=all_numeric_categerical, cutoff=5000 yield columns=293, ACC=0.7694296499301929, in duration=16.8224s
model_type=linear_regression rowset_label=full_country_United States, columnset_label=all_numeric_categerical, cutoff=2500 yield columns=362, ACC=0.8322560628781219, in duration=0.7799s
model_type=random_forest rowset_label=full_country_United States, columnset_label=all_numeric_categerical, cutoff=2500 yield columns=362, ACC=0.8317906820414706, in duration=10.486s


/opt/conda/lib/python3.9/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


model_type=linear_svm rowset_label=full_country_United States, columnset_label=all_numeric_categerical, cutoff=2500 yield columns=362, ACC=0.7892858989606495, in duration=18.5463s
model_type=linear_regression rowset_label=full_country_United States, columnset_label=all_numeric_categerical, cutoff=1000 yield columns=422, ACC=0.8340658772428771, in duration=0.9947s
model_type=random_forest rowset_label=full_country_United States, columnset_label=all_numeric_categerical, cutoff=1000 yield columns=422, ACC=0.8270851646931072, in duration=11.0652s


...

/opt/conda/lib/python3.9/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


model_type=linear_svm rowset_label=full_country_Australia, columnset_label=all_numeric_categerical, cutoff=500 yield columns=443, ACC=0.975076270748229, in duration=17.4108s
model_type=linear_regression rowset_label=full_country_Australia, columnset_label=all_numeric_categerical, cutoff=250 yield columns=450, ACC=0.9816433114431977, in duration=0.9524s
model_type=random_forest rowset_label=full_country_Australia, columnset_label=all_numeric_categerical, cutoff=250 yield columns=450, ACC=0.9816433114431977, in duration=9.0933s


/opt/conda/lib/python3.9/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


model_type=linear_svm rowset_label=full_country_Australia, columnset_label=all_numeric_categerical, cutoff=250 yield columns=450, ACC=0.9659754899426031, in duration=17.2424s
model_type=linear_regression rowset_label=full_country_Australia, columnset_label=all_numeric_categerical, cutoff=100 yield columns=452, ACC=0.9816433114431977, in duration=0.9413s
model_type=random_forest rowset_label=full_country_Australia, columnset_label=all_numeric_categerical, cutoff=100 yield columns=452, ACC=0.9816433114431977, in duration=9.0978s
model_type=linear_svm rowset_label=full_country_Australia, columnset_label=all_numeric_categerical, cutoff=100 yield columns=452, ACC=0.9751279797300791, in duration=16.7751s


/opt/conda/lib/python3.9/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


In [6]:
# result_df = pd.DataFrame(data=results).set_index(['rowset_label', 'columnset_label', 'country', 'cutoff'])
result_df = pd.DataFrame(data=results).set_index(['country', 'cutoff'])
for to_round in ['TPR', 'TNR', 'PPV', 'NPV', 'ACC', 'duration']:
    result_df[to_round] = result_df[to_round].map(lambda x: round(x,3))

result_df.drop(columns=['X_columns', 'coefficients']).sort_values('ACC')

with open('./pickles/latest_results.pkl', 'wb') as f:
    pickle.dump(result_df, f)

cutoff_string = '-'.join(map(str, cutoffs))
dataset_name = 'full_country_row_sample_rate_%s_country_min_sample_cutoff_%s_cutoffs_%s_results' % (row_sample_rate, country_min_sample_cutoff, cutoff_string)
with open('./pickles/%s.pkl' % dataset_name, 'wb') as f:
    pickle.dump(result_df, f)


In [7]:
script_end = time.time()
print('script duration %ss' % round(script_end - start, 3))

script duration 1676.288s
